# Sequence data preprocessing

In [1]:
import pandas as pd
import pyreadr
import numpy as np
import torch
import pickle
from os import listdir
from os.path import isfile, join

In [2]:
patient = pyreadr.read_r('../ServerData_13Oct2020/d_patient_overall.Rdata')
practice = pyreadr.read_r('../ServerData_13Oct2020/d_practice.Rdata')

/opt/conda/envs/rapids/lib/python3.10/site-packages/pyreadr/_pyreadr_parser.py:233: RuntimeWarning: invalid value encountered in cast
  df[colname] = df[colname].values.astype("datetime64[D]").astype(datetime)
/opt/conda/envs/rapids/lib/python3.10/site-packages/pandas/core/tools/datetimes.py:557: RuntimeWarning: invalid value encountered in cast
  arr, tz_parsed = tslib.array_with_unit_to_datetime(arg, unit, errors=errors)
/opt/conda/envs/rapids/lib/python3.10/site-packages/pandas/core/tools/datetimes.py:557: RuntimeWarning: invalid value encountered in multiply
  arr, tz_parsed = tslib.array_with_unit_to_datetime(arg, unit, errors=errors)
/opt/conda/envs/rapids/lib/python3.10/site-packages/pyreadr/_pyreadr_parser.py:233: RuntimeWarning: invalid value encountered in cast
  df[colname] = df[colname].values.astype("datetime64[D]").astype(datetime)
/opt/conda/envs/rapids/lib/python3.10/site-packages/pyreadr/_pyreadr_parser.py:233: RuntimeWarning: invalid value encountered in cast
  df[col

In [3]:
patient = patient['d_patient_overall']
practice = practice['d_practice']

#Age in 2016-01-01
patient['age'] = patient.year_of_birth.apply(lambda x: 2016-x)

In [4]:
patient = patient[['patid', 'practice_id', 'age']].merge(practice[['practice_id', 'Country']], how='left', on='practice_id')

In [5]:
# Outcomes data
outcomes = pd.read_csv("../FinalData/cleaned_outcomes_11072023.csv")

# Clinical

In [ ]:
#load clinical information
path = '../ServerData_13Oct2020/'
clinical_files = [join(path, f) for f in listdir(path) if (isfile(join(path, f))) & ('f_clinical_part' in f)]
clinical = pyreadr.read_r('../ServerData_13Oct2020/f_clinical_part1.Rdata')
clinical = clinical['f_clinical_part']
for file in clinical_files[1:3]:
    temp = pyreadr.read_r(file)
    temp = temp['f_clinical_part']
    print(temp.shape)
    clinical = pd.concat([clinical, temp])
    clinical.reset_index(drop=True, inplace=True)

In [ ]:
clinical.shape

In [ ]:
#data selection 
clinical = clinical.dropna(subset=['code_id'])
clinical['event_date'] = pd.to_datetime(clinical['event_date'])
clinical = clinical.loc[(clinical['event_date'] >= '2016-01-01') & (clinical['event_date'] < '2017-01-01')]

In [ ]:
print(clinical.patid.unique().shape)
print(clinical.shape)

In [ ]:
#vocab and code2idx generation
vocab = clinical.code_id.unique().tolist()
idx = range(1, len(vocab)+1)

code2idx = dict(zip(vocab, idx))
idx2code = dict(zip(idx, vocab))

code2idx['PAD'] = 0
idx2code[0] = 'PAD'
VOCAB_SIZE = len(code2idx)
print('code2idx Size: {}'.format(len(code2idx)))
print('idx2code Size: {}'.format(len(idx2code)))

In [ ]:
#extract year, month, day from event date
clinical['day'] = clinical.apply(lambda x: str(x['event_date'].day), axis=1)
clinical['month'] = clinical.apply(lambda x: str(x['event_date'].month), axis=1)
clinical['year'] = clinical.apply(lambda x: str(x['event_date'].year), axis=1)

In [ ]:
print(clinical.patid.unique().shape)
print(clinical.shape)

In [ ]:
clinical.reset_index(drop=True, inplace=True)
clinical['read_code_seq'] = clinical.sort_values(['event_date'], ascending=True).groupby('patid')['code_id'].transform(lambda x: ', '.join(x))
event_data_seq = clinical.sort_values(['event_date']).groupby('patid').agg({'day': lambda x: x.tolist(),
                                                          'month': lambda x: x.tolist(),
                                                          'year': lambda x: x.tolist()}).reset_index()
clinical = clinical.drop_duplicates(subset=['patid']).reset_index(drop=True)     

In [ ]:
print(clinical.patid.unique().shape)
print(clinical.shape)

In [ ]:
data = clinical[['patid', 'read_code_seq']].merge(event_data_seq, how='left', on='patid')
data = data.merge(outcomes[['patid', 'outcome_3months','outcome_combined_6months','outcome_combined_12months',
                           'outcome_combined_24months']], how='inner', on='patid')
data.outcome_combined_12months.value_counts()

In [ ]:
print(data.patid.unique().shape)
print(data.shape)

In [ ]:
data['read_code_seq'] = data['read_code_seq'].apply(lambda x: x.strip('""').split(', '))
data['length_read_code_seq'] = data['read_code_seq'].apply(lambda x: len(x))
data = data[data.length_read_code_seq > 10]

In [ ]:
data_therapy[data_therapy.length_read_code_seq>50].shape

In [ ]:
print(data.patid.unique().shape)
print(data.shape)

In [ ]:
#padding at the beginning of the list
max_seq = 500
def make_uniform_data(x):
    if len(x) < max_seq:
        pads = ['PAD'] * (max_seq - len(x))
        return pads + x
    elif len(x) > max_seq:
        x = x[len(x)-max_seq:]
        return x
    else:
        return x

In [ ]:
data['read_code_seq_padded'] = data['read_code_seq'].apply(lambda x: make_uniform_data(x))
data['read_code_seq_padded_idx'] = data['read_code_seq_padded'].apply(lambda x: [code2idx.get(key) for key in x])
data = data.merge(patient[['patid', 'Country', 'age']], how='left', on='patid')
data = data.drop_duplicates(subset=['patid']).reset_index(drop=True)     

In [ ]:
print(data.patid.unique().shape)
print(data.shape)

In [ ]:
#SAVE all important materials
pickle.dump(code2idx, open('../SeqModel/code2idx.sav', 'wb'))
pickle.dump(idx2code, open('../SeqModel/idx2code.sav', 'wb'))
pickle.dump(data, open('../SeqModel/data.sav', 'wb'))
data.to_csv('../SeqModel/seqDataRaw.csv', index_label=False, index=False)

In [ ]:
data.length_read_code_seq.hist(bins=50)

In [ ]:
data[data.length_read_code_seq > 300].shape

In [ ]:
# a = pickle.load(open('../SeqModel/data.sav', 'rb'))

# Medication

In [ ]:
#add paractice information

#load clinical information
path = '../ServerData_13Oct2020/'
therapy_files = [join(path, f) for f in listdir(path) if (isfile(join(path, f))) & ('f_therapy_part' in f)]
therapy = pyreadr.read_r('../ServerData_13Oct2020/f_therapy_part1.Rdata')
therapy = therapy['f_therapy_part']
for file in therapy_files[1:3]:
    temp = pyreadr.read_r(file)
    temp = temp['f_therapy_part']
    print(temp.shape)
    therapy = pd.concat([therapy, temp])
    therapy.reset_index(drop=True, inplace=True)

In [ ]:
#data selection 
therapy = therapy.dropna(subset=['code_id'])
therapy['event_date'] = pd.to_datetime(therapy['event_date'])
therapy = therapy.loc[(therapy['event_date'] >= '2016-01-01') & (therapy['event_date'] < '2017-01-01')]

In [ ]:
therapy.patid.unique().shape

In [ ]:
therapy.shape

In [ ]:
#vocab and code2idx generation
vocab_therapy = therapy.code_id.unique().tolist()
idx_therapy = range(1, len(vocab_therapy)+1)

code2idx_therapy = dict(zip(vocab_therapy, idx_therapy))
idx2code_therapy = dict(zip(idx_therapy, vocab_therapy))

code2idx_therapy['PAD'] = 0
idx2code_therapy[0] = 'PAD'
VOCAB_SIZE = len(code2idx_therapy)
print('code2idx Size: {}'.format(len(code2idx_therapy)))
print('idx2code Size: {}'.format(len(idx2code_therapy)))
# print(code2idx)

In [ ]:
#extract year, month, day from event date
therapy['day'] = therapy.apply(lambda x: str(x['event_date'].day), axis=1)
therapy['month'] = therapy.apply(lambda x: str(x['event_date'].month), axis=1)
therapy['year'] = therapy.apply(lambda x: str(x['event_date'].year), axis=1)

In [ ]:
therapy['read_code_seq'] = therapy.sort_values(['event_date'], ascending=True).groupby('patid')['code_id'].transform(lambda x: ', '.join(x))
event_data_seq_therapy = therapy.sort_values(['event_date']).groupby('patid').agg({'day': lambda x: x.tolist(),
                                                          'month': lambda x: x.tolist(),
                                                          'year': lambda x: x.tolist()}).reset_index()
therapy = therapy.drop_duplicates(subset=['patid']).reset_index(drop=True)     

In [ ]:
therapy.patid.unique().shape

In [ ]:
therapy.shape

In [ ]:
data_therapy = therapy[['patid', 'read_code_seq']].merge(event_data_seq_therapy, how='left',  on='patid')
data_therapy = data_therapy.merge(outcomes[['patid', 'outcome_3months','outcome_combined_6months','outcome_combined_12months',
                           'outcome_combined_24months']], how='inner', on='patid')
data_therapy.outcome_combined_12months.value_counts()

In [ ]:
data_therapy.patid.unique().shape

In [ ]:
data_therapy.shape

In [ ]:
data_therapy['read_code_seq'] = data_therapy['read_code_seq'].apply(lambda x: x.strip('""').split(', '))
data_therapy['length_read_code_seq'] = data_therapy['read_code_seq'].apply(lambda x: len(x))
data_therapy = data_therapy[data_therapy.length_read_code_seq > 10]

In [ ]:
#padding at the beginning of the list
max_seq = 50
def make_uniform_data(x):
    if len(x) < max_seq:
        pads = ['PAD'] * (max_seq - len(x))
        return pads + x
    elif len(x) > max_seq:
        x = x[len(x)-max_seq:]
        return x
    else:
        return x

In [ ]:
data_therapy['read_code_seq_padded'] = data_therapy['read_code_seq'].apply(lambda x: make_uniform_data(x))
data_therapy['read_code_seq_padded_idx'] = data_therapy['read_code_seq_padded'].apply(lambda x: [code2idx_therapy.get(key) for key in x])
data_therapy = data_therapy.merge(patient[['patid', 'Country', 'age']], how='left', on='patid')
data_therapy = data_therapy.drop_duplicates(subset=['patid']).reset_index(drop=True)     

In [ ]:
print(data_therapy.patid.unique().shape)
print(data_therapy.shape)

In [ ]:
#SAVE all important materials
pickle.dump(code2idx_therapy, open('../SeqModel/code2idx_therapy.sav', 'wb'))
pickle.dump(idx2code_therapy, open('../SeqModel/idx2code_therapy.sav', 'wb'))
pickle.dump(data_therapy, open('../SeqModel/data_therapy.sav', 'wb'))
data_therapy.to_csv('../SeqModel/seqData_therapyRaw.csv', index_label=False, index=False)

In [ ]:
data_therapy.length_read_code_seq.hist()

In [ ]:
data_therapy[data_therapy.length_read_code_seq<50].shape

In [ ]:
data_therapy.shape

# Combine clinical + therapy

In [ ]:
#load clinical information
path = '../ServerData_13Oct2020/'
clinical_files = [join(path, f) for f in listdir(path) if (isfile(join(path, f))) & ('f_clinical_part' in f)]
clinical = pyreadr.read_r('../ServerData_13Oct2020/f_clinical_part1.Rdata')
clinical = clinical['f_clinical_part']
for file in clinical_files[1:3]:
    temp = pyreadr.read_r(file)
    temp = temp['f_clinical_part']
    print(temp.shape)
    clinical = pd.concat([clinical, temp])
    clinical.reset_index(drop=True, inplace=True)
    
#data selection 
clinical = clinical.dropna(subset=['code_id'])
clinical['event_date'] = pd.to_datetime(clinical['event_date'])
clinical = clinical.loc[(clinical['event_date'] >= '2016-01-01') & (clinical['event_date'] < '2017-01-01')]

In [ ]:
#add paractice information

#load clinical information
path = '../ServerData_13Oct2020/'
therapy_files = [join(path, f) for f in listdir(path) if (isfile(join(path, f))) & ('f_therapy_part' in f)]
therapy = pyreadr.read_r('../ServerData_13Oct2020/f_therapy_part1.Rdata')
therapy = therapy['f_therapy_part']
for file in therapy_files[1:3]:
    temp = pyreadr.read_r(file)
    temp = temp['f_therapy_part']
    print(temp.shape)
    therapy = pd.concat([therapy, temp])
    therapy.reset_index(drop=True, inplace=True)
    
#data selection 
therapy = therapy.dropna(subset=['code_id'])
therapy['event_date'] = pd.to_datetime(therapy['event_date'])
therapy = therapy.loc[(therapy['event_date'] >= '2016-01-01') & (therapy['event_date'] < '2017-01-01')]

In [ ]:
all_raw_data = pd.concat([clinical[['patid', 'event_date', 'code_id']],
           therapy[['patid', 'event_date', 'code_id']]])
all_raw_data.reset_index(drop=True, inplace=True)

In [ ]:
#vocab and code2idx generation
vocab_all = all_raw_data.code_id.unique().tolist()
idx_all = range(1, len(vocab_all)+1)

code2idx_all = dict(zip(vocab_all, idx_all))
idx2code_all = dict(zip(idx_all, vocab_all))

code2idx_all['PAD'] = 0
idx2code_all[0] = 'PAD'
VOCAB_SIZE= len(code2idx_all)
print('code2idx Size: {}'.format(len(code2idx_all)))
print('idx2code Size: {}'.format(len(idx2code_all)))
# print(code2idx)

In [ ]:
#extract year, month, day from event date
all_raw_data['day'] = all_raw_data.apply(lambda x: str(x['event_date'].day), axis=1)
all_raw_data['month'] = all_raw_data.apply(lambda x: str(x['event_date'].month), axis=1)
all_raw_data['year'] = all_raw_data.apply(lambda x: str(x['event_date'].year), axis=1)

In [ ]:
all_raw_data['read_code_seq'] = all_raw_data.sort_values(['event_date'], ascending=True).groupby('patid')['code_id'].transform(lambda x: ', '.join(x))
event_data_seq_all = all_raw_data.sort_values(['event_date']).groupby('patid').agg({'day': lambda x: x.tolist(),
                                                          'month': lambda x: x.tolist(),
                                                          'year': lambda x: x.tolist()}).reset_index()
all_raw_data = all_raw_data.drop_duplicates(subset=['patid']).reset_index(drop=True)     

In [ ]:
data_all = all_raw_data[['patid', 'read_code_seq']].merge(event_data_seq_all, how='left',  on='patid')
data_all = data_all.merge(outcomes[['patid', 'outcome_3months','outcome_combined_6months','outcome_combined_12months',
                           'outcome_combined_24months']], how='inner', on='patid')
data_all.outcome_combined_12months.value_counts()

In [ ]:
print(data_all.shape)
print(data_all.patid.unique().shape)

In [ ]:
data_all['read_code_seq'] = data_all['read_code_seq'].apply(lambda x: x.strip('""').split(', '))
data_all['length_read_code_seq'] = data_all['read_code_seq'].apply(lambda x: len(x))
data_all = data_all[data_all.length_read_code_seq > 10]

In [ ]:
data_all[data_all.length_read_code_seq>200].shape

In [ ]:
data_all.shape

In [ ]:
#padding at the beginning of the list
max_seq = 150
def make_uniform_data(x):
    if len(x) < max_seq:
        pads = ['PAD'] * (max_seq - len(x))
        return pads + x
    elif len(x) > max_seq:
        x = x[len(x)-max_seq:]
        return x
    else:
        return x

In [ ]:
data_all['read_code_seq_padded'] = data_all['read_code_seq'].apply(lambda x: make_uniform_data(x))
data_all['read_code_seq_padded_idx'] = data_all['read_code_seq_padded'].apply(lambda x: [code2idx_all.get(key) for key in x])
data_all = data_all.merge(patient[['patid', 'Country', 'age']], how='left', on='patid')
data_all = data_all.drop_duplicates(subset=['patid']).reset_index(drop=True)     

In [ ]:
print(data_all.shape)
print(data_all.patid.unique().shape)

In [ ]:
#SAVE all important materials
pickle.dump(code2idx_all, open('../SeqModel/code2idx_all.sav', 'wb'))
pickle.dump(idx2code_all, open('../SeqModel/idx2code_all.sav', 'wb'))
pickle.dump(data_all, open('../SeqModel/data_all.sav', 'wb'))
data_all.to_csv('../SeqModel/seqData_all.csv', index_label=False, index=False)

# Get all data

In [6]:
#load clinical and therapy information

#function for sequence padding
def make_uniform_data(x):
    max_seq = 150
    if len(x) < max_seq:
        pads = ['PAD'] * (max_seq - len(x))
        return pads + x
    elif len(x) > max_seq:
        x = x[len(x)-max_seq:]
        return x
    else:
        return x


#=========================================================================#

#function to generate the sequence of clinical and therapy data
def process_sequence(clinical_file, therapy_file):
    clinical = pyreadr.read_r(clinical_file)
    clinical = clinical['f_clinical_part']
    therapy_files = [join(path, f) for f in listdir(path) if (isfile(join(path, f))) & ('f_therapy_part' in f)]
    therapy = pyreadr.read_r(therapy_file)
    therapy = therapy['f_therapy_part']

    #data selection 2016&2017
    clinical = clinical.dropna(subset=['code_id'])
    clinical['event_date'] = pd.to_datetime(clinical['event_date'])
    clinical = clinical.loc[(clinical['event_date'] >= '2016-01-01') & (clinical['event_date'] < '2017-01-01')]

    therapy = therapy.dropna(subset=['code_id'])
    therapy['event_date'] = pd.to_datetime(therapy['event_date'])
    therapy = therapy.loc[(therapy['event_date'] >= '2016-01-01') & (therapy['event_date'] < '2017-01-01')]


    #concat 2 data
    temp_all_raw_data = pd.concat([clinical[['patid', 'event_date', 'code_id']],
               therapy[['patid', 'event_date', 'code_id']]])
    temp_all_raw_data.reset_index(drop=True, inplace=True)

    #create vocab list
    temp_vocab = temp_all_raw_data.code_id.unique().tolist()

    #extract year, month, day from event date (if needed)
    temp_all_raw_data['day'] = temp_all_raw_data.apply(lambda x: str(x['event_date'].day), axis=1)
    temp_all_raw_data['month'] = temp_all_raw_data.apply(lambda x: str(x['event_date'].month), axis=1)
    temp_all_raw_data['year'] = temp_all_raw_data.apply(lambda x: str(x['event_date'].year), axis=1)

    #checkpoint 1
    print("checkpoint1")
    print(temp_all_raw_data.shape)
    print(temp_all_raw_data.patid.unique().shape)

    #concat all read code per patient per event date (sorted by date, oldest to newest)
    temp_all_raw_data['read_code_seq'] = temp_all_raw_data.sort_values(['event_date'], ascending=True).groupby('patid')['code_id'].transform(lambda x: ', '.join(x))
    temp_event_data_seq_all = temp_all_raw_data.sort_values(['event_date']).groupby('patid').agg({'day': lambda x: x.tolist(),
                                                              'month': lambda x: x.tolist(),
                                                              'year': lambda x: x.tolist()}).reset_index()
    temp_all_raw_data = temp_all_raw_data.drop_duplicates(subset=['patid']).reset_index(drop=True)     
    temp_all_raw_data.reset_index(drop=True, inplace=True)

    #checkpoint 2
    print("checkpoint2")
    print(temp_all_raw_data.shape)
    print(temp_all_raw_data.patid.unique().shape)


    #merge sequence data with outcomes
    temp_data_all = temp_all_raw_data[['patid', 'read_code_seq']].merge(temp_event_data_seq_all, how='left',  on='patid')

    #checkpoint 3
    print("checkpoint3")
    print(temp_data_all.shape)
    print(temp_data_all.patid.unique().shape)


    # get sequence length per patient and only keep patient with >10 read codes
    temp_data_all['read_code_seq'] = temp_data_all['read_code_seq'].apply(lambda x: x.strip('""').split(', '))
    temp_data_all['length_read_code_seq'] = temp_data_all['read_code_seq'].apply(lambda x: len(x))
    temp_data_all = temp_data_all[temp_data_all.length_read_code_seq > 10]

    #checkpoint 4
    print("checkpoint4")
    print(temp_data_all.shape)
    print(temp_data_all.patid.unique().shape)

    #padded data
    temp_data_all['read_code_seq_padded'] = temp_data_all['read_code_seq'].apply(lambda x: make_uniform_data(x))

    #checkpoint 5
    print("checkpoint5")
    print(temp_data_all.shape)
    print(temp_data_all.patid.unique().shape)
    
    return temp_data_all, temp_vocab


In [8]:
path = '../ServerData_13Oct2020/'
clinical_files = [join(path, f) for f in listdir(path) if (isfile(join(path, f))) & ('f_clinical_part' in f)]
therapy_files = [join(path, f) for f in listdir(path) if (isfile(join(path, f))) & ('f_therapy_part' in f)]
vocab_all = []
cols = ['patid', 'read_code_seq', 'day', 'month', 'year', 'length_read_code_seq',
           'read_code_seq_padded']
data_all = pd.DataFrame(columns=cols)

nFiles=len(therapy_files)
batch=10

for i in range(0,1):
    startIndexFile = i*batch
    endIndexFile = startIndexFile+batch
    if endIndexFile > nFiles:
        endIndexFile = nFiles
    print("#######################################################")
    print(startIndexFile, endIndexFile)
    data_chunk = pd.DataFrame(columns=cols)
    vocab_chunk = [] 
    for clinical_file, therapy_file in zip(clinical_files[startIndexFile:endIndexFile], therapy_files[startIndexFile:endIndexFile]):
        print("===================================================")
        print(clinical_file, therapy_file)
        #combine data_all
        temp_data, temp_vocab = process_sequence(clinical_file, therapy_file)
        data_chunk = pd.concat([data_chunk, temp_data])
        data_chunk.reset_index(drop=True, inplace=True)
        vocab_chunk = vocab_chunk + temp_vocab
    data_all = pd.concat([data_all, data_chunk])
    data_all.reset_index(drop=True, inplace=True)
    vocab_all = vocab_all + vocab_chunk
    pickle.dump(vocab_all, open('../SeqModel/vocab_all_big.sav', 'wb'))
    pickle.dump(data_all, open('../SeqModel/data_all_big.sav', 'wb'))
    
    
    
    



#######################################################
0 10
../ServerData_13Oct2020/f_clinical_part1.Rdata ../ServerData_13Oct2020/f_therapy_part1.Rdata


/opt/conda/envs/rapids/lib/python3.10/site-packages/pyreadr/_pyreadr_parser.py:233: RuntimeWarning: invalid value encountered in cast
  df[colname] = df[colname].values.astype("datetime64[D]").astype(datetime)
/opt/conda/envs/rapids/lib/python3.10/site-packages/pandas/core/tools/datetimes.py:557: RuntimeWarning: invalid value encountered in cast
  arr, tz_parsed = tslib.array_with_unit_to_datetime(arg, unit, errors=errors)
/opt/conda/envs/rapids/lib/python3.10/site-packages/pandas/core/tools/datetimes.py:557: RuntimeWarning: invalid value encountered in multiply
  arr, tz_parsed = tslib.array_with_unit_to_datetime(arg, unit, errors=errors)
/opt/conda/envs/rapids/lib/python3.10/site-packages/pandas/core/tools/datetimes.py:557: RuntimeWarning: invalid value encountered in cast
  arr, tz_parsed = tslib.array_with_unit_to_datetime(arg, unit, errors=errors)
/opt/conda/envs/rapids/lib/python3.10/site-packages/pandas/core/tools/datetimes.py:557: RuntimeWarning: invalid value encountered in mu

checkpoint1
(1258874, 6)
(9787,)
checkpoint2
(9787, 7)
(9787,)
checkpoint3
(9787, 5)
(9787,)
checkpoint4
(9207, 6)
(9207,)
checkpoint5
(9207, 7)
(9207,)
../ServerData_13Oct2020/f_clinical_part10.Rdata ../ServerData_13Oct2020/f_therapy_part10.Rdata


/opt/conda/envs/rapids/lib/python3.10/site-packages/pyreadr/_pyreadr_parser.py:233: RuntimeWarning: invalid value encountered in cast
  df[colname] = df[colname].values.astype("datetime64[D]").astype(datetime)
/opt/conda/envs/rapids/lib/python3.10/site-packages/pandas/core/tools/datetimes.py:557: RuntimeWarning: invalid value encountered in cast
  arr, tz_parsed = tslib.array_with_unit_to_datetime(arg, unit, errors=errors)
/opt/conda/envs/rapids/lib/python3.10/site-packages/pandas/core/tools/datetimes.py:557: RuntimeWarning: invalid value encountered in multiply
  arr, tz_parsed = tslib.array_with_unit_to_datetime(arg, unit, errors=errors)
/opt/conda/envs/rapids/lib/python3.10/site-packages/pandas/core/tools/datetimes.py:557: RuntimeWarning: invalid value encountered in cast
  arr, tz_parsed = tslib.array_with_unit_to_datetime(arg, unit, errors=errors)
/opt/conda/envs/rapids/lib/python3.10/site-packages/pandas/core/tools/datetimes.py:557: RuntimeWarning: invalid value encountered in mu

checkpoint1
(1299245, 6)
(9811,)
checkpoint2
(9811, 7)
(9811,)
checkpoint3
(9811, 5)
(9811,)
checkpoint4
(9301, 6)
(9301,)
checkpoint5
(9301, 7)
(9301,)
../ServerData_13Oct2020/f_clinical_part11.Rdata ../ServerData_13Oct2020/f_therapy_part11.Rdata


...

checkpoint1
(1306752, 6)
(9845,)
checkpoint2
(9845, 7)
(9845,)
checkpoint3
(9845, 5)
(9845,)
checkpoint4
(9313, 6)
(9313,)
checkpoint5
(9313, 7)
(9313,)
../ServerData_13Oct2020/f_clinical_part17.Rdata ../ServerData_13Oct2020/f_therapy_part17.Rdata


/opt/conda/envs/rapids/lib/python3.10/site-packages/pyreadr/_pyreadr_parser.py:233: RuntimeWarning: invalid value encountered in cast
  df[colname] = df[colname].values.astype("datetime64[D]").astype(datetime)
/opt/conda/envs/rapids/lib/python3.10/site-packages/pandas/core/tools/datetimes.py:557: RuntimeWarning: invalid value encountered in cast
  arr, tz_parsed = tslib.array_with_unit_to_datetime(arg, unit, errors=errors)
/opt/conda/envs/rapids/lib/python3.10/site-packages/pandas/core/tools/datetimes.py:557: RuntimeWarning: invalid value encountered in multiply
  arr, tz_parsed = tslib.array_with_unit_to_datetime(arg, unit, errors=errors)
/opt/conda/envs/rapids/lib/python3.10/site-packages/pandas/core/tools/datetimes.py:557: RuntimeWarning: invalid value encountered in cast
  arr, tz_parsed = tslib.array_with_unit_to_datetime(arg, unit, errors=errors)
/opt/conda/envs/rapids/lib/python3.10/site-packages/pandas/core/tools/datetimes.py:557: RuntimeWarning: invalid value encountered in mu

checkpoint1
(1282650, 6)
(9796,)
checkpoint2
(9796, 7)
(9796,)
checkpoint3
(9796, 5)
(9796,)
checkpoint4
(9264, 6)
(9264,)
checkpoint5
(9264, 7)
(9264,)
../ServerData_13Oct2020/f_clinical_part18.Rdata ../ServerData_13Oct2020/f_therapy_part18.Rdata


/opt/conda/envs/rapids/lib/python3.10/site-packages/pyreadr/_pyreadr_parser.py:233: RuntimeWarning: invalid value encountered in cast
  df[colname] = df[colname].values.astype("datetime64[D]").astype(datetime)
/opt/conda/envs/rapids/lib/python3.10/site-packages/pandas/core/tools/datetimes.py:557: RuntimeWarning: invalid value encountered in cast
  arr, tz_parsed = tslib.array_with_unit_to_datetime(arg, unit, errors=errors)
/opt/conda/envs/rapids/lib/python3.10/site-packages/pandas/core/tools/datetimes.py:557: RuntimeWarning: invalid value encountered in multiply
  arr, tz_parsed = tslib.array_with_unit_to_datetime(arg, unit, errors=errors)
/opt/conda/envs/rapids/lib/python3.10/site-packages/pandas/core/tools/datetimes.py:557: RuntimeWarning: invalid value encountered in cast
  arr, tz_parsed = tslib.array_with_unit_to_datetime(arg, unit, errors=errors)
/opt/conda/envs/rapids/lib/python3.10/site-packages/pandas/core/tools/datetimes.py:557: RuntimeWarning: invalid value encountered in mu

checkpoint1
(1256231, 6)
(9785,)
checkpoint2
(9785, 7)
(9785,)
checkpoint3
(9785, 5)
(9785,)
checkpoint4
(9156, 6)
(9156,)
checkpoint5
(9156, 7)
(9156,)


In [10]:
data_all.shape

(92960, 7)

In [11]:
#vocab and code2idx generation
vocab_all_big = vocab_all
vocab_all_big=list(dict.fromkeys(vocab_all_big)) #remove duplicate
idx_all = range(1, len(vocab_all_big)+1)
code2idx_all = dict(zip(vocab_all_big, idx_all))
idx2code_all = dict(zip(idx_all, vocab_all_big))
code2idx_all['PAD'] = 0
idx2code_all[0] = 'PAD'
VOCAB_SIZE= len(code2idx_all)
print('code2idx Size: {}'.format(len(code2idx_all)))
print('idx2code Size: {}'.format(len(idx2code_all)))
# print(code2idx)

# padded to index, and merge with patient data (age, Country)
data_all_big = data_all
data_all_big['read_code_seq_padded_idx'] = data_all_big['read_code_seq_padded'].apply(lambda x: [code2idx_all.get(key) for key in x])
data_all_big = data_all_big.merge(outcomes[['patid', 'outcome_3months','outcome_combined_6months','outcome_combined_12months',
                               'outcome_combined_24months', ]], how='inner', on='patid')
data_all_big = data_all_big.merge(patient[['patid', 'Country', 'age']], how='left', on='patid')
data_all_big = data_all_big.drop_duplicates(subset=['patid']).reset_index(drop=True)     


code2idx Size: 45211
idx2code Size: 45211


In [12]:
data_all_big.shape

(75059, 14)

In [14]:
data_all_big.patid.unique().shape

(75059,)

In [16]:
len(vocab_all_big)

45210

In [15]:
#SAVE all important materials
pickle.dump(code2idx_all, open('../SeqModel/code2idx_all_big.sav', 'wb'))
pickle.dump(idx2code_all, open('../SeqModel/idx2code_all_big.sav', 'wb'))
pickle.dump(data_all_big, open('../SeqModel/data_all_big.sav', 'wb'))
# data_all.to_csv('../SeqModel/seqData_all.csv', index_label=False, index=False)

In [17]:
outcomes

,patid,outcome_3months,outcome_6months,outcome_9months,outcome_12months,outcome_15months,outcome_18months,outcome_21months,outcome_24months,outcome_combined_6months,outcome_combined_9months,outcome_combined_12months,outcome_combined_15months,outcome_combined_18months,outcome_combined_24months
0,43231452,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,43206365,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,43203606,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,43117348,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,43289035,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
577957,66462185,0,0,0,0,0,0,0,0,0,0,0,0,0,0
577958,66462486,0,0,0,0,0,0,0,0,0,0,0,0,0,0
577959,65728252,0,0,0,0,0,0,0,0,0,0,0,0,0,0
577960,66464892,0,0,0,0,0,0,0,0,0,0,0,0,0,0


# END